# loading data

In [2]:
import struct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

In [4]:
raw_train = read_idx("./data/train-images.idx3-ubyte")
X = np.reshape(raw_train, (60000, 28*28))#train_data
y = read_idx("./data/train-labels.idx1-ubyte")#train_label

raw_test = read_idx("./data/t10k-images.idx3-ubyte")
test_data = np.reshape(raw_test, (10000, 28*28))
test_label = read_idx("./data/t10k-labels.idx1-ubyte")

/home/ryu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


# preprocessing

In [5]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()

X = mm.fit_transform(X)

In [6]:
print(X.shape)
print(y.shape)

(60000, 784)
(60000,)


In [80]:
init_w=np.random.rand(len(y), 1)
for i in range(len(y)):
    c=y*init_w

MemoryError: Unable to allocate array with shape (60000, 60000) and data type float64

In [24]:
def img(row, data):
    image = np.zeros((28,28))
    for i in range(0,28):
        for j in range(0,28):
            pix = 28*i+j
            image[i,j] = data[row, pix]
    plt.imshow(image, cmap = 'gray')
    plt.show()
    print (data[row,0])

In [49]:
#for i in range(0,28):
    
print(X[1][4])

0.0


In [66]:
a,b=np.shape(X)
w=np.random.rand(b,2)

print(w[:])

[[0.96274918 0.06093266]
 [0.97743482 0.77509122]
 [0.91058599 0.64947179]
 ...
 [0.44509887 0.92948797]
 [0.68890782 0.85983389]
 [0.54956329 0.28028116]]


In [75]:
b = np.dot(X, w)
a = b
print(a)   

[[57.46716273 51.66963117]
 [65.80249496 59.18657626]
 [39.25348771 41.42311678]
 ...
 [47.81839745 45.99045469]
 [40.73770211 38.9850228 ]
 [46.06091187 43.42109188]]


In [ ]:
def one_number(labels, number):
	return (labels == number)*1

In [ ]:
label_ones = one_number(y_train, 2)

In [ ]:
def update(weights, data_point, labels, alpha=.1):
	predicted = predict(data_point, weights)
#for i in range(0, len(weights)):
	weight_temp = np.zeros(np.shape(weights))
	weight_temp[:,0] = alpha*(labels-predicted)*data_point
	return weight_temp+weights

In [ ]:
np.shape(update(weights, X_train[3], label_ones[3]))

In [ ]:
def train_perceptron(data, labels, weights, alpha = .1, iterations = 100):
	for j in range(0, iterations):
		for i in range(0, len(data)):
			weights = update(weights, data[i], labels[i], alpha)
	return weights

In [ ]:
weights_new = train_perceptron(X_train, label_ones, weights, alpha = .1, iterations = 50)

In [ ]:
label_ones = one_number(y_train, 3)
weights_new = train_perceptron(X_train, label_ones, weights, alpha = .01, iterations =6)

In [ ]:
def test_perceptron_f(data, labels, weights):
    a,b = np.shape(data)
    predicted = predict(data, weights)
    correct = (predicted==labels)*1==1
    true_pos = np.sum((labels==1)*(correct))
    true_neg = np.sum((labels==0)*(correct))
    total_pos = np.sum(labels)
    total_neg = a-total_pos
    tp_p = true_pos/float(np.sum(labels))
    tn_p = true_neg/float(a- np.sum(labels))
    return true_pos, true_neg, tp_p, tn_p, total_pos, total_neg
    
a,b,c,d,e,f = test_perceptron_f(X_train, label_ones, weights_new)

In [ ]:
print c, d

In [ ]:
for i in range(0,10):
	num = i
	label_ones = one_number(y_train, num)
	weights_new = train_perceptron(X_train, label_ones, weights, alpha = .1, iterations =5)
	label_test = one_number(y_test, num)
	a = test_perceptron_f(X_test, label_test, weights_new)

In [ ]:
def all_numbers(data,labels):
	c,d = np.shape(data)
	w = create_weights(data)
	weights = []
	for i in range(0,  len(np.unique(labels))):
		z = one_number(labels, i)
		a = train_perceptron(data, z, w, .1, 4)
		weights.append(a[:,0])
	return np.asarray(weights)

In [ ]:
w = all_numbers(X_train, y_train)

In [ ]:
print np.shape(w)

In [ ]:
def one_all(data, weights):
	a = np.dot(data,np.transpose(weights))
	b = len(np.shape(data))
	if b == 1:
		return np.argmax(a)
	return np.argmax(a, axis=1)


def test_all(data, labels, weights):
	a, b = np.shape(labels)
	predicted = one_all(data, weights)
	correct = predicted == labels[:,0]
	accuracy = np.sum(correct)/float(a)
	return accuracy

print test_all(X_train, y_train, w)

In [ ]:
lbls = one_number(y_train, 2)
lbls_t = one_number(y_test, 2)


In [ ]:
alpha = []
t = .000007
for i in range(0,1):
    alpha.append(t)
    t = t/10.0
True_pos = []
True_neg = []
w = create_weights(X_train)
for a in alpha:
    weights = w
    print a
    temp_p = []
    temp_n = []
    for j in range(0,200):
        for k in range(0, len(X_train)):
            weights = update(weights, X_train[k], lbls[k], a)
        f,b,c,d,e,g = test_perceptron_f(X_test, lbls_t, weights)
        temp_p.append(c)
        temp_n.append(d)
    True_pos.append(temp_p)
    True_neg.append(temp_n)
    print temp_p

https://github.com/zswarth/MNIST-Classification/blob/master/MNIST.ipynb